# Limpieza

## Parámetros

In [1]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Contests/Ecobici/data'

## Importar

In [2]:
from ecobici.models import EcoBici

# Importar Clase con todos sus métodos y atributos
eb = EcoBici(BASE_DIR)
print(eb)

44 archivos en:
/Users/efraflores/Desktop/EF/Contests/Ecobici/data


In [ ]:
# Importar todos los archivos csv
df = eb.read_raw_files(from_year=2018, to_year=2021, index=['Ciclo_Estacion_Retiro','fecha'], columns='fecha_hour_range', values='n', aggfunc='count', fill_value=0)

## Transformar

In [ ]:
# Cambiar la estructura para comparar ventanas de tiempo de N días anteriores
X, y = eb.ecobici_shifted(df, id_cols=['Ciclo_Estacion_Retiro'], shifts=range(41))

## Modelo

In [20]:
# Importar desde BASE_DIR
data = eb.get_csv('EcoBici_shifted.csv', just_name=True, add_subfolder=True, subfolder_name='transformed').set_index(['Ciclo_Estacion_Retiro','fecha'])

Archivo con nombre EcoBici_shifted.csv fue encontrado en:
/Users/efraflores/Desktop/EF/Contests/Ecobici/data/transformed
Con 616938 renglones y 289 columnas


In [21]:
# Obtener la lista de las columnas de todos los días previos
prev = data.head(1).filter(regex='_\d+').columns.tolist()

# Y aquellas originales, sin escalonar
actual = [x for x in data.columns if x not in prev]

In [ ]:
# Ordena las columnas
data = data[actual+prev].copy()

In [22]:
# Seleccionar los datos para construir f(X)=y
X = data[prev].copy()
y = data[actual].sum(axis=1).values

X.sample()

,,00 a 09_01,00 a 09_02,00 a 09_03,00 a 09_04,00 a 09_05,00 a 09_06,00 a 09_07,00 a 09_08,00 a 09_09,00 a 09_10,...,>= 21_31,>= 21_32,>= 21_33,>= 21_34,>= 21_35,>= 21_36,>= 21_37,>= 21_38,>= 21_39,>= 21_40
Ciclo_Estacion_Retiro,fecha,,,,,,,,,,,,,,,,,,,,,
24.0,2021-08-15,5.0,9.0,8.0,10.0,5.0,9.0,6.0,8.0,3.0,4.0,...,3.0,6.0,4.0,7.0,4.0,0.0,6.0,7.0,7.0,2.0


### Entrenamiento

In [11]:
model_linreg, scores_linreg, coef_linreg = eb.train_model(X, y)

Test score:	88.70%
Train score:	88.65%


In [12]:
from sklearn.ensemble import RandomForestRegressor

model_forest, scores_forest, coef_forest = eb.train_model(X, y, model=RandomForestRegressor)

Test score:	88.03%
Train score:	98.26%


In [13]:
from sklearn.neural_network import MLPRegressor

model_neural, scores_neural, coef_neural = eb.train_model(X, y, model=MLPRegressor)

Test score:	87.65%
Train score:	96.40%


### Variables más importantes

In [14]:
model = model_linreg
coef = coef_linreg

In [15]:
n_vars = 10

coef.head(n_vars//2).append(coef.tail(n_vars//2)).style.format("{:.2f}")

,Peso
Variable,
00 a 09_01,4.33
18 a 20_01,3.87
00 a 09_07,3.56
15 a 17_01,2.76
00 a 09_35,2.48
00 a 09_08,-0.89
15 a 17_39,-0.95
15 a 17_09,-1.23
00 a 09_22,-1.33


### Real vs estimado

In [23]:
eb.plot_real_vs_est(X, y, model, id_col='Ciclo_Estacion_Retiro')

## Preservación del Modelo

In [ ]:
eb.save_model({'pronostico':model, 'pronostico_cols':X.columns}, 'ecobici_modelo_pronostico')